In [1]:
import timeit
import numpy as np
import torch

from openmm import *
from openmm.unit import *
from openmm.app import *

import pickle


def investigate(obj):
    return list(filter(lambda x: '__' not in x, dir(obj)))


def size(obj):
    return len(pickle.dumps(obj))


R = 8.314e-3
kBT = R * 300


def state2energy(state):
    energy = state.getPotentialEnergy().value_in_unit(
    kilojoule / mole) / kBT

    # get forces
    force = -state.getForces(asNumpy=True).value_in_unit(
        kilojoule / mole / nanometer) / kBT

    return energy, force

parameter_topology = AmberPrmtopFile('task/energies/data/aldp/aldp.prmtop')

input_coordinate = AmberInpcrdFile('task/energies/data/aldp/aldp.crd')
input_coordinate_numpy = np.array(input_coordinate.positions.value_in_unit(nanometer)).reshape(66)
batch_input_numpy = np.tile(input_coordinate_numpy, (300, 1))

system = parameter_topology.createSystem(implicitSolvent=OBC1, constraints=None, nonbondedCutoff=None, hydrogenMass=None)

In [ ]:
# CUDA + serial

context = Context(
    system,
    LangevinIntegrator(
        300 * kelvin,
        1. / picosecond,
        1. * femtosecond
    ),
    Platform.getPlatformByName('CUDA'),
)

def energy_evaluation():
    context.setPositions(input_coordinate.positions)
    state = context.getState(getEnergy=True, getForces=True)

timeit.timeit("energy_evaluation()", globals=globals(), number=300 * 1000)

In [3]:
# multiprocessing CPU - 6 worker

import multiprocessing as mp

def varinit():
    global context
    context = Context(
        system,
        LangevinIntegrator(
            300 * kelvin,
            1. / picosecond,
            1. * femtosecond
        ),
        Platform.getPlatformByName('Reference'),
    )



def batch_proc(molecule):
    molecule = molecule.reshape(-1, 3)
    context.setPositions(molecule)
    state = context.getState(getEnergy=True, getForces=True)

    state2energy(state)


pool = mp.Pool(1, varinit)


# 300 evaluations
def energy_evaluation():
    pool.map(batch_proc, batch_input_numpy)

timeit.timeit("energy_evaluation()", globals=globals(), number=1000)

433.05045987293124

In [ ]:
# multiprocessing CPU - 48 worker

import multiprocessing as mp

def varinit():
    global context
    context = Context(
        system,
        LangevinIntegrator(
            300 * kelvin,
            1. / picosecond,
            1. * femtosecond
        ),
        Platform.getPlatformByName('Reference'),
    )



def batch_proc(molecule):
    molecule = molecule.reshape(-1, 3)
    context.setPositions(molecule)
    state = context.getState(getEnergy=True, getForces=True)

    state2energy(state)


pool = mp.Pool(4, varinit)


# 300 evaluations
def energy_evaluation():
    pool.map(batch_proc, batch_input_numpy)

timeit.timeit("energy_evaluation()", globals=globals(), number=100)

In [9]:
# CUDA + MP

import multiprocessing as mp

num_workers = 1
input_coordinate = AmberInpcrdFile('task/energies/data/aldp/aldp.crd')
input_coordinate_numpy = np.array(input_coordinate.positions.value_in_unit(nanometer)).reshape(66)
batch_input_numpy = np.tile(input_coordinate_numpy, (300, 1))

def varinit():
    global context

    context = Context(
        system,
        LangevinIntegrator(
            300 * kelvin,
            1. / picosecond,
            1. * femtosecond
        ),
        Platform.getPlatformByName('CUDA'),
    )


def batch_proc(molecule):
    global context

    context.setPositions(molecule)
    state = context.getState(getEnergy=True, getForces=True)

    energy, force = state2energy(state)
    return energy, force


pool = mp.Pool(num_workers, varinit)


def energy_evaluation():
    global batch_input_numpy

    batch_input_numpy = batch_input_numpy.reshape(-1, 22, 3)

    results = pool.map(batch_proc, batch_input_numpy)
    return results

timeit.timeit("energy_evaluation()", globals=globals(), number=1000)

51.580193776637316

In [ ]:
# CUDA + MP + 6 workers


from concurrent.futures import ThreadPoolExecutor
import queue

num_workers = 1

contexts = [
    Context(
        system,
        LangevinIntegrator(
            300 * kelvin,
            1. / picosecond,
            1. * femtosecond
        ),
        Platform.getPlatformByName('CUDA'),
    )
    for _ in range(num_workers)
]

context_indices = list(range(num_workers))

# Create a multiprocessing queue and add contexts to it
context_idx_queue = queue.Queue(maxsize=num_workers)

for idx in context_indices:
    context_idx_queue.put(idx)

def task(molecule):
    molecule = molecule.reshape(-1, 3)

    idx = context_idx_queue.get()
    context = contexts[idx]

    context.setPositions(molecule)
    state = context.getState(getEnergy=True, getForces=True)

    context_idx_queue.put(idx)  # Put the context back into the queue

    return state


def energy_evaluation():
    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        futures = executor.map(task, batch_input_numpy)
    return futures

timeit.timeit("energy_evaluation()", globals=globals(), number=1000)